# Training Time

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from slice_generator import slice_generator

In [2]:
print("tensorflow version:",tf.__version__)

tensorflow version: 2.2.0-dlenv


## Model variables

In [3]:
frames = 1
channels = 1
pixels_x = 21
pixels_y = 21

## Load the model

In [4]:
model_name = 'convlstm_'+str(frames)+'f_'+str(channels)+'c_'+str(pixels_x)+'x_'+str(pixels_y)+'y'
print('model name:',model_name)

model name: convlstm_1f_1c_21x_21y


In [5]:
model = tf.keras.models.load_model('../models/'+model_name+'.h5')
model.summary()

Model: "convlstm_1f_1c_21x_21y"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 1, 4, 21, 21)      2016      
_________________________________________________________________
batch_normalization (BatchNo (None, 1, 4, 21, 21)      4         
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 1, 2, 21, 21)      440       
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 2, 21, 21)      4         
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 1, 1, 21, 21)      16        
_________________________________________________________________
batch_normalization_2 (Batch (None, 1, 1, 21, 21)      4         
Total params: 2,484
Trainable params: 2,478
Non-trainable params: 6
__________________________________________

## Prepare for training

In [6]:
train_file_path = "../data/train"
valid_file_path = "../data/validate"
vars_           = ['t2m']
proc_type       = "conv_lstm"
# 3 years of training data = 
train_steps = 3 * 365 * 24 / frames
# 1 year of validation data = 
valid_steps = 1 * 365 * 24 / frames

In [7]:
slice_train = slice_generator(img_dir=train_file_path, slice_size=frames, vars_=vars_,
                     proc_type=proc_type, pixels_x=pixels_x, pixels_y=pixels_y, debug=False
                     )
slice_val = slice_generator(img_dir=valid_file_path, slice_size=frames, vars_=vars_,
                     proc_type=proc_type, pixels_x=pixels_x, pixels_y=pixels_y, debug=False
                     )

## Train

In [10]:
epochs = 5

In [11]:
history = model.fit(
    slice_train,
    steps_per_epoch = train_steps,
    epochs = epochs,
    verbose = 1,
    shuffle = False,
    initial_epoch = 0,
    validation_steps = valid_steps,
    validation_data = slice_val,
    )

Epoch 1/5
26280/26280 [==============================] - 1053s 40ms/step - loss: 26975.5391 - mean_absolute_error: 287.7577 - val_loss: 23106.8789 - val_mean_absolute_error: 287.5146
Epoch 2/5
26280/26280 [==============================] - 1041s 40ms/step - loss: 22706.1797 - mean_absolute_error: 287.7526 - val_loss: 21724.7852 - val_mean_absolute_error: 287.5079
Epoch 3/5
26280/26280 [==============================] - 1022s 39ms/step - loss: 21506.7207 - mean_absolute_error: 287.7470 - val_loss: 21782.4648 - val_mean_absolute_error: 287.5012
Epoch 4/5
26280/26280 [==============================] - 1029s 39ms/step - loss: 19970.2422 - mean_absolute_error: 287.7414 - val_loss: 17441.5723 - val_mean_absolute_error: 287.4960
Epoch 5/5
26280/26280 [==============================] - 1040s 40ms/step - loss: 19688.8398 - mean_absolute_error: 287.7368 - val_loss: 24797.7949 - val_mean_absolute_error: 287.4942


## Save trained model

In [12]:
tf.keras.models.save_model(
    model = model,
    filepath = '../models/t_'+model_name+'.h5',
    overwrite=True,
    include_optimizer=True,
    save_format='tf',
    signatures=None
)